In [1]:
import pandas as pd
from gurobipy import *
import numpy as np

## load data

In [2]:
data = pd.read_excel('焚化廠資料.xlsx')
pd.set_option('display.max_rows', None)
data.columns=['name', 'city', 'year', 'input_other', 'input_labor', 'input_garbage', 'fixed_input_1', 'fixed_input_2', 'output', 'undesirable_output']
data = data.sort_values(by = 'name', ignore_index = True)

## constant

In [23]:
names = np.unique(data['name'])
grouped_year = data.groupby(data.year)

N = range(names.size)
years = range(2011, 2021)
input_name = ['input_other', 'input_labor', 'input_garbage']
fixed_input_name = ['fixed_input_1', 'fixed_input_2']

## NL simulation

In [24]:
record_NT = {}
for t in years:
    record_NT[t] = {}

for t in years:
    
    DATA = grouped_year.get_group(t)
    DATA = DATA.sort_values(by = 'name', ignore_index = True)
    
    X_input = np.array(DATA[input_name]).T
    X_fixed_input = np.array(DATA[fixed_input_name]).T
    Y = np.array(DATA['output'])
    U = np.array(DATA['undesirable_output'])
    
    for j0 in N:
        
        M = Model('NT')
        M.Params.LogToConsole = 0
        
        _lambda = M.addVars(N, lb = 0, ub = 1, name='lambda')

        theta = M.addVar(lb = 0, name='theta')

        w = M.addVars(N, lb = 0, ub = 1, name='w')

        M.update()

        M.setObjective(theta * Y[j0], GRB.MAXIMIZE)
        
        M.addConstrs(
            quicksum(X_input[i][j] * _lambda[j] for j in N) - X_input[i][j0] <= 0
            for i in range(len(input_name))
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * _lambda[j] for j in N) - X_fixed_input[i][j0] <= 0
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(Y[j] * _lambda[j] for j in N) - theta * Y[j0] >= 0
        )
        
        M.addConstr(
            quicksum(_lambda[j] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j] for j in N) - X_fixed_input[i][j0] >= 0
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(U[j] * w[j] for j in N) - theta * U[j0] <= 0
        )
        
        M.addConstr(
            quicksum(w[j] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j] for j in N)
            - quicksum(X_fixed_input[i][j] * _lambda[j] for j in N)
            == 0
            for i in range(len(fixed_input_name))
        )
        
        M.optimize()
        
        record_NT[t][DATA['name'][j0]] = {
            'objval': M.objVal,
            'original output': Y[j0],
            'theta': [v.x for v in M.getVars() if v.varName == 'theta'],
            'val': [{'name': v.varName, 'value': v.x} for v in M.getVars()]
        }

In [25]:
x = [[record_NT[v][j]['objval'] for j in record_NT[v]] for v in record_NT]
sum(sum(np.array(x)))

33085301.991909444

In [26]:
total_objvalue = 0
years = list(range(2011,2021))
for name in names:
    for year in years:
        total_objvalue += record_NT[year][name]['objval']

In [27]:
total_objvalue 

33085301.991909463

## CIT simulation

In [28]:
record_CIT = {}

for t in years:
    
    DATA = grouped_year.get_group(t)
    DATA = DATA.sort_values(by = 'name', ignore_index = True)
    
    X_input = np.array(DATA[input_name]).T
    X_fixed_input = np.array(DATA[fixed_input_name]).T
    Y = np.array(DATA['output'])
    U = np.array(DATA['undesirable_output'])
    
    M = Model('CIT')
    M.Params.LogToConsole = 0
    
    _lambda = M.addVars(N, N, lb = 0, ub = 1, name='lambda')
    
    theta = M.addVars(N, lb = 0, name='theta')
    
    w = M.addVars(N, N, lb = 0, ub = 1, name='w')
    
    delta = M.addVars(N, lb = 0, name='delta')
    
    M.update()
    
    M.setObjective(
        quicksum(theta[k] * Y[k] for k in N),
        GRB.MAXIMIZE
    )
    
    for n in N:
        
        M.addConstrs(
            quicksum(X_input[i][j] * _lambda[n,j] for j in N) <= X_input[i][n]
            for i in range(len(input_name))
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * _lambda[n,j] for j in N) <= X_fixed_input[i][n]
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(Y[j] * _lambda[n,j] for j in N) >= theta[n] * Y[n]
        )
        
        M.addConstr(
            quicksum(_lambda[n,j] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j,n] for j in N) >= X_fixed_input[i][n]
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(U[j] * w[j,n] for j in N) <= delta[n] * U[n]
        )
        
        M.addConstr(
            quicksum(w[j,n] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j,n] for j in N)
            - quicksum(X_fixed_input[i][j] * _lambda[n,j] for j in N)
            == 0
            for i in range(len(fixed_input_name))
        )
        
    M.addConstr(
        quicksum(delta[k] * U[k] for k in N)
        <= quicksum(U[k] for k in N)
    )
        
    M.optimize()

    record_CIT[t] = {
        'objval': M.objVal,
        'original output': sum(Y),
        'theta': [v.x for v in M.getVars() if 'theta' in v.varName],
        'delta': [v.x for v in M.getVars() if 'delta' in v.varName],
        'val': [{'name': v.varName, 'value': v.x} for v in M.getVars()]
    }

In [29]:
x = [record_CIT[v]['objval'] for v in record_CIT]
sum(x)

33085301.991909448

## CIIT simulation

In [31]:
record_CIIT = {}

M = Model('CIT')
M.Params.LogToConsole = 0

_lambda = M.addVars(N, N, years, lb = 0, ub = 1, name='lambda')

theta = M.addVars(N, years, lb = 0, name='theta')

w = M.addVars(N, N, years, lb = 0, ub = 1, name='w')

delta = M.addVars(N, years, lb = 0, name='delta')

udo = M.addVars(years, lb = 0, name='udo')
nudo = M.addVars(years, lb = 0, name='nudo')

M.update()

M.setObjective(
    quicksum(quicksum(theta[k,t] * data[data['name'] == names[k]][data['year'] == t]['output'] for k in N) for t in years),
    GRB.MAXIMIZE
)

for t in years:
    
    DATA = grouped_year.get_group(t)
    DATA = DATA.sort_values(by = 'name', ignore_index = True)
    
    X_input = np.array(DATA[input_name]).T
    X_fixed_input = np.array(DATA[fixed_input_name]).T
    Y = np.array(DATA['output'])
    U = np.array(DATA['undesirable_output'])
    
    
    
    for n in N:
        
        M.addConstrs(
            quicksum(X_input[i][j] * _lambda[j,n,t] for j in N) <= X_input[i][n]
            for i in range(len(input_name))
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * _lambda[j,n,t] for j in N) <= X_fixed_input[i][n]
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(Y[j] * _lambda[j,n,t] for j in N) >= theta[n,t] * Y[n]
        )
        
        M.addConstr(
            quicksum(_lambda[j,n,t] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j,n,t] for j in N) >= X_fixed_input[i][n]
            for i in range(len(fixed_input_name))
        )
        
        M.addConstr(
            quicksum(U[j] * w[j,n,t] for j in N) <= delta[n,t] * U[n]
        )
        
        M.addConstr(
            quicksum(w[j,n,t] for j in N) == 1
        )
        
        M.addConstrs(
            quicksum(X_fixed_input[i][j] * w[j,n,t] for j in N)
            - quicksum(X_fixed_input[i][j] * _lambda[j,n,t] for j in N)
            == 0
            for i in range(len(fixed_input_name))
        )
        
    M.addConstr(
        quicksum(U[k] for k in N) == udo[t]
    )
    
    M.addConstr(
        quicksum(U[k] * delta[k,t] for k in N) == nudo[t]
    )
        
M.addConstr(
    quicksum(nudo[t] for t in years)
    <= quicksum(udo[t] for t in years)
)

M.optimize()

record_CIIT[t] = {
    'objval': M.objVal,
    'original output': sum(Y),
    'theta': [v.x for v in M.getVars() if 'theta' in v.varName],
    'delta': [v.x for v in M.getVars() if 'delta' in v.varName],
    'val': [{'name': v.varName, 'value': v.x} for v in M.getVars()]
}

<ipython-input-31-da08d5a8ac20>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quicksum(quicksum(theta[k,t] * data[data['name'] == names[k]][data['year'] == t]['output'] for k in N) for t in years),


In [32]:
record_CIIT

{2020: {'objval': 33085301.991909463,
  'original output': 3088359.0400000005,
  'theta': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0169711141158508,
   1.0491765365323262,
   1.074933077382943,
   1.0371247106155126,
   1.037062356593412,
   1.1929683585116875,
   1.2328062440566272,
   1.1660732957410032,
   1.0173366683743568,
   1.0655799930146348,
   1.0,
   1.0,
   1.1583615645263645,
   1.1515951627989705,
   1.1818464690252828,
   1.1770493048822228,
   1.164709607659409,
   1.2410747835511504,
   1.1996591324108838,
   1.301775369182497,
   1.1952856638865086,
   1.1666924107175554,
   1.2018089132900434,
   1.1563229562711375,
   1.2094422538126877,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.69083042498384